# 杭州萧山区项目
## 本地排放清单预处理`Local Emission Inventory`

---
*@author: Evan*\
*@date: 2023-07-19*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import os

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../../src/')
from namelist import *
import findpoint as fp

创建网格变量

In [2]:
grid = xr.open_dataset(datadir+'GRIDCRO2D_2022234.nc')
lat = grid.LAT[0,0,:,:]
lon = grid.LON[0,0,:,:]

gridfile = xr.Dataset(
    data_vars = dict(
        ShapeVar = (['y','x'],np.zeros_like(lat),{'long name':'not-used variable'})
    ),
    coords=dict(
        latitude = (['y','x'],lat.data),
        longitude = (['y','x'],lon.data)
    )
)
gridfile

<xarray.Dataset>
Dimensions:    (y: 138, x: 135)
Coordinates:
    latitude   (y, x) float32 28.29 28.29 28.29 28.29 ... 31.89 31.89 31.89
    longitude  (y, x) float32 117.9 117.9 117.9 118.0 ... 122.3 122.3 122.4
Dimensions without coordinates: y, x
Data variables:
    ShapeVar   (y, x) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

读取本地清单面源分类

In [3]:
ef = pd.ExcelFile(emisarea)
ef.sheet_names

['机动车-不分',
 '非道路-船舶和港口机械',
 '非道路-工程机械',
 '非道路-农业机械',
 '扬尘源-道路扬尘',
 '扬尘源-施工扬尘',
 '扬尘源-土壤扬尘',
 '非工业溶剂使用-农药使用',
 '非工业溶剂使用-建筑涂料使用',
 '非工业溶剂使用-其他',
 '生物质-露天焚烧',
 '生物质-家用秸秆薪柴',
 '农业源-畜禽养殖',
 '农业源-氮肥施用',
 '农业源-土壤本底',
 '农业源-固氮植物',
 '农业源-秸秆堆肥',
 '农业源-人体粪便',
 '其他排放源-餐饮',
 '其他排放源-民用燃烧',
 '面源合计']

设定本地清单与MEIC源分类的对应关系

In [4]:
sma = pd.read_excel(secmap).groupby('SourceType').get_group('area')
sma

,MEIC,LocalPrimarySource,LocalSecondarySource,SourceType
6,Transportation,机动车,不分,area
7,Transportation,非道路移动源,船舶和港口机械,area
8,Transportation,非道路移动源,工程机械,area
9,Transportation,非道路移动源,农业机械,area
10,Transportation,扬尘源,道路扬尘,area
11,Transportation,扬尘源,施工扬尘,area
13,Agriculture,扬尘源,土壤扬尘,area
14,Agriculture,非工业溶剂使用,农药使用,area
15,Agriculture,生物质,露天焚烧,area
16,Agriculture,生物质,家用秸秆薪柴,area


根据对应关系，将本地源分类映射到MEIC的五类中

In [5]:
sma_grouped = sma.groupby('MEIC')
sections = list(sma_grouped.groups.keys()) # ['Agriculture', 'Industry', 'Power', 'Residential', 'Transportation']
species = ['SO2','NOx','CO','PM10','PM25','VOCs','NH3','BC','OC']

df_target = {}
for sec in sections:
    dfs = {}
    for sheet_name in ef.sheet_names:
        parts = sheet_name.split('-')
        primary = parts[0]
        secondary = parts[1] if len(parts) > 1 else None
        
        if secondary in sma_grouped.get_group(sec)['LocalSecondarySource'].values:
            print(f'{primary}-{secondary} --> {sec}')
            
            current_df = ef.parse(sheet_name).drop(['FID_1'],axis=1)
            coord_cols = ['CenLon', 'CenLat']
        
            if primary not in dfs:
                dfs[primary] = current_df
            else:
                merged_df = pd.merge(dfs[primary], current_df, on=coord_cols,
                             how='outer', suffixes=('_x', '_y'))

                for specie in species:
                    merged_df[specie] = merged_df[f'{specie}_x'] + merged_df[f'{specie}_y']
                    merged_df = merged_df.drop(columns=[f'{specie}_x',f'{specie}_y'],axis=1)
        
                dfs[primary] = merged_df
    
    df_target[sec] = pd.concat(dfs,axis=0).reset_index(drop=True)

扬尘源-土壤扬尘 --> Agriculture
非工业溶剂使用-农药使用 --> Agriculture
生物质-露天焚烧 --> Agriculture
生物质-家用秸秆薪柴 --> Agriculture
农业源-畜禽养殖 --> Agriculture
农业源-氮肥施用 --> Agriculture
农业源-土壤本底 --> Agriculture
农业源-固氮植物 --> Agriculture
农业源-秸秆堆肥 --> Agriculture
农业源-人体粪便 --> Agriculture
非工业溶剂使用-建筑涂料使用 --> Residential
非工业溶剂使用-其他 --> Residential
其他排放源-餐饮 --> Residential
机动车-不分 --> Transportation
非道路-船舶和港口机械 --> Transportation
非道路-工程机械 --> Transportation
非道路-农业机械 --> Transportation
扬尘源-道路扬尘 --> Transportation
扬尘源-施工扬尘 --> Transportation


将清单依照经纬度写入网格点，保存为nc文件

In [6]:
for sec in sections:
    temp = fp.assign_values_to_grid(df_target[sec],gridfile,'CenLon','CenLat',species)
    temp.to_netcdf(f'D:/Download/{sec}.nc')
    print(f'{sec} finished!')

Complete SO2
Complete NOx
Complete CO
Complete PM10
Complete PM25
Complete VOCs
Complete NH3
Complete BC
Complete OC
Agriculture finished!
Complete SO2
Complete NOx
Complete CO
Complete PM10
Complete PM25
Complete VOCs
Complete NH3
Complete BC
Complete OC
Residential finished!
Complete SO2
Complete NOx
Complete CO
Complete PM10
Complete PM25
Complete VOCs
Complete NH3
Complete BC
Complete OC
Transportation finished!
